## Домашнее задание 3: Contingency Table

### О задании

1) Consider three nominal features (one of them, not more, may be taken from nominal features in your data).

2) Build two contingency tables over them: present a conditional frequency table and Quetelet relative index tables. Make comments on relations between categories of the common (to both tables) feature and two others.

3) Compute and visualize the chi-square-summary_Quetelet_index over both tables.

4) Comment on the meaning of the values in the data analysis context.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Загрузим данные

In [23]:
df = pd.read_csv('../../data/Vehicles0515.csv')
df = df.replace(-1, np.NaN)
df = df.dropna(subset=['Age_of_Driver', 'Driving_Experience'])
df.head()

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Driving_Experience,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type
0,200501BS00001,1,9,0.0,18,0,0.0,0.0,0.0,0.0,...,1.0,15.0,2,74.0,10.0,NaN,NaN,NaN,7.0,1.0
1,200501BS00002,1,11,0.0,4,0,3.0,0.0,0.0,0.0,...,1.0,1.0,1,42.0,7.0,8268.0,2.0,3.0,NaN,NaN
2,200501BS00003,1,11,0.0,17,0,0.0,0.0,4.0,0.0,...,1.0,1.0,1,35.0,6.0,8300.0,2.0,5.0,2.0,1.0
3,200501BS00003,2,9,0.0,2,0,0.0,0.0,0.0,0.0,...,1.0,15.0,1,62.0,9.0,1762.0,1.0,6.0,1.0,1.0
4,200501BS00004,1,9,0.0,18,0,0.0,0.0,0.0,0.0,...,1.0,15.0,2,49.0,8.0,1769.0,1.0,4.0,2.0,1.0


Добавим 2 номинальных признака

In [24]:
df = df.assign(Vehicle_Type_LEQ15=df.Vehicle_Type.copy())
df.loc[df["Vehicle_Type"]<=15, "Vehicle_Type_LEQ15"] = 1
df.loc[df["Vehicle_Type"]>15, "Vehicle_Type_LEQ15"] = 0

df = df.assign(Age_of_Driver_LEQ40=df.Age_of_Driver.copy())
df.loc[df["Age_of_Driver"]<=40, "Age_of_Driver_LEQ40"] = 1
df.loc[df["Age_of_Driver"]>40, "Age_of_Driver_LEQ40"] = 0

In [25]:
df.columns

Index(['Accident_Index', 'Vehicle_Reference', 'Vehicle_Type',
       'Towing_and_Articulation', 'Vehicle_Manoeuvre',
       'Vehicle_Location-Restricted_Lane', 'Junction_Location',
       'Skidding_and_Overturning', 'Hit_Object_in_Carriageway',
       'Vehicle_Leaving_Carriageway', 'Hit_Object_off_Carriageway',
       '1st_Point_of_Impact', 'Was_Vehicle_Left_Hand_Drive?',
       'Journey_Purpose_of_Driver', 'Sex_of_Driver', 'Age_of_Driver',
       'Driving_Experience', 'Engine_Capacity_(CC)', 'Propulsion_Code',
       'Age_of_Vehicle', 'Driver_IMD_Decile', 'Driver_Home_Area_Type',
       'Vehicle_Type_LEQ15', 'Age_of_Driver_LEQ40'],
      dtype='object')

В качестве 3-х номинальных признаков выберем: "Sex_of_Driver", "Vehicle_Type_LEQ15", "Age_of_Driver_LEQ40"

In [53]:
df.groupby(["Sex_of_Driver"])["Vehicle_Reference"].value_counts()

Sex_of_Driver  Vehicle_Reference
1              1                    18409
               2                     9441
               3                     1099
               4                      201
               5                       47
               6                       17
               7                        6
               8                        2
2              1                     6295
               2                     3147
               3                      411
               4                       80
               5                       14
               7                        2
               8                        2
3              1                       14
               2                       11
               3                       11
Name: Vehicle_Reference, dtype: int64